In [180]:
#! pip install cloud-sql-python-connector["pg8000"]

In [181]:
import io
from datetime import datetime, time, timedelta

import pandas as pd
import pg8000
import pytz
import requests
import sqlalchemy
from google.cloud import bigquery
from google.cloud.sql.connector import Connector
from sqlalchemy import text

In [182]:
fmt = '%Y-%m-%dT%H:%M:%SZ'
cl = pytz.timezone("America/Santiago")
utc = pytz.utc

In [ ]:
def create_connection():
    POSTGRES_CONNECTION_NAME = "conexion-datos-rdf:us-central1:rdf-db"
    POSTGRES_USER = "postgres"
    POSTGRES_PASS = "n3FLOvsqa8nqtjym"
    POSTGRES_DB = "rdf-db"

    connector = Connector()

    def init_connection_engine() -> sqlalchemy.engine.Engine:
        def getconn() -> pg8000.dbapi.Connection:
            conn: pg8000.dbapi.Connection = connector.connect(
                POSTGRES_CONNECTION_NAME,
                "pg8000",
                user=POSTGRES_USER,
                password=POSTGRES_PASS,
                db=POSTGRES_DB,
            )
            return conn

        engine = sqlalchemy.create_engine(
            "postgresql+pg8000://",
            creator=getconn,
        )
        engine.dialect.description_encoding = None
        return engine

    db = init_connection_engine()

    return db

In [184]:
df = pd.read_csv("C:Users/alopez/Desktop/Consolidado_encuestas.csv")
#df['proveedores_consumo'].apply(lambda x : x.split())

In [185]:
df['region_declarada'].value_counts()

Región Metropolitana de Santiago                    147
Región de Valparaíso                                  5
Región del Maule                                      2
Región de La Araucanía                                2
Región de Magallanes y la Antártica Chilena           1
Región de Los Lagos                                   1
Región del Libertador General Bernardo O’Higgins      1
Región del Biobío                                     1
Name: region_declarada, dtype: int64

In [186]:
df['region_declarada'] = df['region_declarada'].replace(['Región Metropolitana de Santiago', 
'Región de Valparaíso', 'Región del Maule', 'Región de La Araucanía', 'Región de Los Lagos',
'Región del Biobío', 'Región del Libertador General Bernardo O’Higgins', 
'Región de Magallanes y la Antártica Chilena' ], ['Metropolitana','Valparaíso', 'Maule', 'Araucanía',
'Los Lagos', 'Biobío', "O'Higgins", 'Magallanes'])

In [187]:
df['region_declarada'].value_counts()

Metropolitana    147
Valparaíso         5
Araucanía          2
Maule              2
Magallanes         1
O'Higgins          1
Biobío             1
Los Lagos          1
Name: region_declarada, dtype: int64

In [188]:
df = df.rename(columns={
    "consumo_od_podcasts": "consumo_od", 
    "proveedores_consumo": "consumo_distribudor", 
    "favoritos_consumo": "consumo_favoritos", 
    "interes_consumo": "consumo_interes"})

In [189]:
df['consumo_od'] = df['consumo_od'].replace(['No', 'Sí'], ['0','1'])
df['consumo_od'] = df['consumo_od'].astype(bool)

In [190]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [191]:
import numpy as np
df['consumo_favoritos'] = df['consumo_favoritos'].str.split(',')
df['consumo_favoritos'] = df['consumo_favoritos'].fillna('').apply(list)
df['consumo_interes'] = df['consumo_interes'].str.split(',')
df['consumo_interes'] = df['consumo_interes'].fillna('').apply(list)
df['consumo_distribudor'] = df['consumo_distribudor'].str.split(',')
df['consumo_distribudor'] = df['consumo_distribudor'].fillna('').apply(list)
df['subcategoria_favoritos'] = df['subcategoria_favoritos'].str.split(',')
df['subcategoria_favoritos'] = df['subcategoria_favoritos'].fillna('').apply(list)
df['categoria_favoritos'] = df['categoria_favoritos'].str.split(',')
df['categoria_favoritos'] = df['categoria_favoritos'].fillna('').apply(list)
df['subcategoria_interes'] = df['subcategoria_interes'].str.split(',')
df['subcategoria_interes'] = df['subcategoria_interes'].fillna('').apply(list)
df['categoria_interes'] = df['categoria_interes'].str.split(',')
df['categoria_interes'] = df['categoria_interes'].fillna('').apply(list)

0                                   []
1                                   []
2                                   []
3                                   []
4                                   []
                    ...               
264      [Música,  Historia,  Ficción]
265             [Historia,  Política ]
266    [Misceláneo,  Humor,  Historia]
267      [Misceláneo,  Humor,  Música]
268                           [Música]
Name: consumo_favoritos, Length: 269, dtype: object

In [193]:
df = df[["id_encuesta","email","datetime","soporte","region_declarada",
"consumo_distribudor","consumo_favoritos","categoria_favoritos",
"subcategoria_favoritos","consumo_interes","categoria_interes", 
"subcategoria_interes", "consumo_od"]]

In [ ]:
df.sample()

In [ ]:
df_final = df.copy()
schema = "usuarios"
table = "encuestas"

columnas = {"id_encuesta": ""}

chunk_size = 32767 // len(df_final.columns)
chunks = (len(df_final) // chunk_size) + 1
print(f"Se subirán {chunks} chunks de con un máximo de {chunk_size} filas")

connection = create_connection()

connection.execute('''TRUNCATE TABLE {0}.{1}'''.format(schema, table))
df_final.to_sql(table, connection, schema=schema, if_exists='append', method='multi', index=False, chunksize=chunk_size)
print("Datos cargados exitosamente a la ficha de los usuarios")

In [153]:
bq_table = "conexion-datos-rdf.encuestas.consolidado"

client = bigquery.Client()

job_config = bigquery.LoadJobConfig(
        schema=[
                bigquery.SchemaField("id_encuesta", bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField("email", bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField("datetime", bigquery.enums.SqlTypeNames.DATETIME),
                bigquery.SchemaField("soporte", bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField("region_declarada", bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField("consumo_distribudor", bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField("consumo_favoritos", bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField("categoria_favoritos", bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField("subcategoria_favoritos", bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField("consumo_interes", bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField("categoria_interes", bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField("subcategoria_interes", bigquery.enums.SqlTypeNames.STRING),
                bigquery.SchemaField("consumo_od", bigquery.enums.SqlTypeNames.BOOLEAN),
                ],
        write_disposition="WRITE_TRUNCATE",
        )
    
# Make an API request.
job = client.load_table_from_dataframe(df, bq_table, job_config=job_config) 

print(job.result())  # Wait for the job to complete.   

LoadJob<project=conexion-datos-rdf, location=US, id=c23060fe-7bca-41b1-a441-fef3da06e3a2>
